In [1]:
# Import modules
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import datetime as dt

import re

import requests
from bs4 import BeautifulSoup as soup

import pyxlsb
from pyxlsb import convert_date

import warnings

%qtconsole

In [2]:
# Define local directories
master_dir = 'C:\\Users\\Bryan\\OneDrive\\02 GitHub\\rig-count_L48\\'
data_raw_dir = 'C:\\Users\\Bryan\\OneDrive\\02 GitHub\\rig-count_L48\\data\\raw\\'
data_interim_dir = 'C:\\Users\\Bryan\\OneDrive\\02 GitHub\\rig-count_L48\\data\\interim\\'
data_external_dir = 'C:\\Users\\Bryan\\OneDrive\\02 GitHub\\rig-count_L48\\data\\external\\'

In [3]:
# Get web files, convert to .xlsb and save
url = 'https://rigcount.bakerhughes.com/na-rig-count'

dinner = soup(requests.get(url).text,'lxml')

# Find the tag with appropriate text, extract URL
file_link = dinner.find('a', text='North America Rotary Rig Count (Jan 2000 - Current)')['href']

res = requests.get(file_link)

file_d_name = 'nam_rig-count'

with open(data_external_dir+file_d_name+'.xlsb','wb') as local:
    for chunk in res.iter_content(chunk_size=128):
        local.write(chunk)

In [4]:
# Read Excel file
filename = 'nam_rig-count'
sheet0 = 'Current Weekly Summary'
sheet1 = 'US Count by Basin'
sheet2 = 'US Oil & Gas Split'
sheet3 = 'US Count by Trajectory'

df = pd.read_excel(data_external_dir + filename+'.xlsb', sheet_name=sheet1, header=9,usecols=range(0,65),engine='pyxlsb')

# Save raw file with date qualifier
file_date = convert_date(df.iloc[1:,0].max()).strftime('%Y-%m-%d')

df.to_excel(data_raw_dir+filename+'_'+file_date+'.xlsx',engine='xlsxwriter')

df.head()

,Unnamed: 0,Ardmore Woodford,Unnamed: 2,Unnamed: 3,Unnamed: 4,Arkoma Woodford,Unnamed: 6,Unnamed: 7,Unnamed: 8,Barnett,...,Unnamed: 55,Unnamed: 56,Others,Unnamed: 58,Unnamed: 59,Unnamed: 60,Total US RigCount,Unnamed: 62,Unnamed: 63,Unnamed: 64
0,Date,Oil,Gas,Misc,Total,Oil,Gas,Misc,Total,Oil,...,Misc,Total,Oil,Gas,Misc,Total,Oil,Gas,Misc,Total
1,40578,1,2,0,3,0,19,0,19,18,...,0,157,180,255,6,441,818,911,10,1739
2,40585,1,3,0,4,0,19,0,19,18,...,0,158,172,248,6,426,805,906,10,1721
3,40592,1,3,0,4,0,19,0,19,18,...,0,158,173,245,6,424,798,905,10,1713
4,40599,1,3,0,4,0,19,0,19,18,...,0,158,168,247,6,421,783,906,10,1699


In [5]:
# Change 'Unnamed' columns to NaN
col_list = df.columns.to_list()

for col in range(0,len(col_list)):
    if 'Unnamed' in col_list[col]:
        col_list[col] = np.nan

df.columns = pd.Series(col_list).fillna(method='ffill').tolist()
        
df.head()

,NaN,Ardmore Woodford,Ardmore Woodford,Ardmore Woodford,Ardmore Woodford,Arkoma Woodford,Arkoma Woodford,Arkoma Woodford,Arkoma Woodford,Barnett,...,Williston,Williston,Others,Others,Others,Others,Total US RigCount,Total US RigCount,Total US RigCount,Total US RigCount
0,Date,Oil,Gas,Misc,Total,Oil,Gas,Misc,Total,Oil,...,Misc,Total,Oil,Gas,Misc,Total,Oil,Gas,Misc,Total
1,40578,1,2,0,3,0,19,0,19,18,...,0,157,180,255,6,441,818,911,10,1739
2,40585,1,3,0,4,0,19,0,19,18,...,0,158,172,248,6,426,805,906,10,1721
3,40592,1,3,0,4,0,19,0,19,18,...,0,158,173,245,6,424,798,905,10,1713
4,40599,1,3,0,4,0,19,0,19,18,...,0,158,168,247,6,421,783,906,10,1699


In [6]:
# Creating new data frame, remove non-oil counts
col_unique = pd.Series(col_list).fillna(method='ffill').unique()[1:]
df_new = pd.DataFrame(df.iloc[1:,0])
df_new.columns = ['date']

for num in range(0,len(col_unique)):
    try:
        df_trans = pd.DataFrame(df.iloc[1:,1]) 
        df_new = pd.concat([df_new,df_trans],axis=1)
        df.pop(col_unique[num])
    
        df_trans = pd.DataFrame()
    except:
        print(col_unique[num])
        print(df.head())
        
print(df_new.head(3))
print(df_new.tail(3))

    date Ardmore Woodford Arkoma Woodford Barnett Cana Woodford DJ-Niobrara  \
1  40578                1               0      18            15          11   
2  40585                1               0      18            12          10   
3  40592                1               0      18            12          10   

  Eagle Ford Fayetteville Granite Wash Haynesville Marcellus Mississippian  \
1         47            0           25           1         0             6   
2         46            0           24           1         0             8   
3         46            0           24           1         0             8   

  Permian Utica Williston Others Total US RigCount  
1     357     0       157    180               818  
2     355     0       158    172               805  
3     347     0       158    173               798  
      date Ardmore Woodford Arkoma Woodford Barnett Cana Woodford DJ-Niobrara  \
499  44064                0               0       0             6           4

In [7]:
# Remove NaN, Convert date column to date time string

warnings.filterwarnings('ignore')

df_new[df_new.loc[:,'date'].isna()].shape     # 2 lines of NaN

df_dropped = df_new.dropna()     # drop NaN values

df_dropped.loc[:,'date'] = df_dropped.loc[:,'date'].apply(convert_date)

# Convert dtypes to numeric
df_dropped.iloc[:,1:] = df_dropped.iloc[:,1:].apply(pd.to_numeric, errors='coerce')

df_dropped = df_dropped.set_index('date').resample('W-FRI').mean()
df_dropped = df_dropped.reset_index()

warnings.filterwarnings('always')

df_dropped.tail()

,date,Ardmore Woodford,Arkoma Woodford,Barnett,Cana Woodford,DJ-Niobrara,Eagle Ford,Fayetteville,Granite Wash,Haynesville,Marcellus,Mississippian,Permian,Utica,Williston,Others,Total US RigCount
494,2020-07-24,1,0,0,5,4,12,0,1,0,0,0,126,0,10,22,181
495,2020-07-31,1,0,0,6,4,11,0,1,0,0,0,124,0,11,22,180
496,2020-08-07,1,0,0,6,4,10,0,1,0,0,0,121,0,11,22,176
497,2020-08-14,0,0,0,6,4,10,0,1,0,0,0,116,0,11,24,172
498,2020-08-21,0,0,0,6,4,9,0,1,0,0,0,126,0,10,27,183


In [8]:
# Save df_dropped to csv
df_dropped.to_csv(data_interim_dir+'nam_rig-count_by-basin.csv',index=False)